In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_scheduler
from torch.utils.data import DataLoader, Dataset
import torch
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from nltk.corpus import wordnet


from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the dataset
file_path = '/content/drive/MyDrive/cleaned_labeled_data_output2.csv'
data = pd.read_csv(file_path)

In [ ]:


# Drop redundant column
if 'Unnamed: 0' in data.columns:
    data.drop(columns=['Unnamed: 0'], inplace=True)

# Clean tweets
def clean_tweet(tweet):
    tweet = re.sub(r"http\S+", "", tweet)  # Remove URLs
    tweet = re.sub(r"[^A-Za-z0-9\s]", "", tweet)  # Remove special characters
    tweet = re.sub(r"\s+", " ", tweet).strip()  # Remove extra spaces
    return tweet

data['tweet'] = data['tweet'].apply(clean_tweet)

# Step 2: Split dataset
train_data, temp_data = train_test_split(data, test_size=0.3, stratify=data['class'], random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, stratify=temp_data['class'], random_state=42)

# Step 3: Balance the dataset (Oversampling Hate Speech class)
hate_speech = train_data[train_data['class'] == 0]
train_data_balanced = pd.concat([
    train_data,
    resample(hate_speech, replace=True, n_samples=1000, random_state=42)
])

# Step 4: Tokenize data
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')  # Replace with ReBERT if available

def tokenize_data(data):
    return tokenizer(
        data['tweet'].tolist(),
        padding='max_length',
        truncation=True,
        max_length=64,  # Reduced sequence length for faster training
        return_tensors='pt'
    )

train_encodings = tokenize_data(train_data_balanced)
val_encodings = tokenize_data(val_data)
test_encodings = tokenize_data(test_data)

# Step 5: Define Dataset class
class HateSpeechDataset(Dataset):
    def __init__(self, tokenized_texts, labels):
        self.tokenized_texts = tokenized_texts
        self.labels = labels.values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.tokenized_texts['input_ids'][idx],
            'attention_mask': self.tokenized_texts['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

train_dataset = HateSpeechDataset(train_encodings, train_data_balanced['class'])
val_dataset = HateSpeechDataset(val_encodings, val_data['class'])
test_dataset = HateSpeechDataset(test_encodings, test_data['class'])

# Step 6: Prepare DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)
test_dataloader = DataLoader(test_dataset, batch_size=16)

# Step 7: Define the model
model = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v3-base', num_labels=3)  # Replace with ReBERT if available

# Step 8: Define optimizer, scheduler, and class weights
optimizer = AdamW(model.parameters(), lr=2e-5)  # Reduced learning rate
scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * 10
)

# Increase weight for Hate Speech
class_weights = torch.tensor([3.0, 1.0, 2.0], dtype=torch.float).to('cuda')
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Step 9: Define the evaluate_model function
def evaluate_model(dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
            labels = batch['labels'].to(device)

            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_preds, all_labels

# Training with Mixed Precision and Early Stopping
from torch.cuda.amp import autocast, GradScaler
scaler = GradScaler()
accumulation_steps = 2
epochs = 10
best_val_f1 = 0
early_stopping_patience = 3
patience_counter = 0

for epoch in range(epochs):
    model.train()
    total_loss = 0
    print(f"Epoch {epoch + 1}/{epochs}")

    for batch_idx, batch in enumerate(tqdm(train_dataloader)):
        inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        with autocast():
            outputs = model(**inputs)
            loss = loss_fn(outputs.logits, labels)
            loss = loss / accumulation_steps

        scaler.scale(loss).backward()

        if (batch_idx + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        total_loss += loss.item() * accumulation_steps

    scheduler.step()
    print(f"Average training loss: {total_loss / len(train_dataloader):.4f}")

    # Validation
    val_preds, val_labels = evaluate_model(val_dataloader)
    val_report = classification_report(
        val_labels,
        val_preds,
        target_names=['Hate Speech', 'Offensive', 'Neutral'],
        output_dict=True
    )
    val_f1 = val_report['Hate Speech']['f1-score']
    print(f"Validation F1 for Hate Speech: {val_f1:.4f}")

    # Early Stopping
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), "best_rebert_model.pth")
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= early_stopping_patience:
        print("Early stopping triggered.")
        break

# Step 10: Load Best Model and Test
model.load_state_dict(torch.load("best_rebert_model.pth"))
test_preds, test_labels = evaluate_model(test_dataloader)
print("Test Metrics:")
print(classification_report(test_labels, test_preds, target_names=['Hate Speech', 'Offensive', 'Neutral']))


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed 

Epoch 1/10


  0%|          | 0/574 [00:00<?, ?it/s]<ipython-input-7-c1faf5791dbd>:121: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 574/574 [01:39<00:00,  5.80it/s]


Average training loss: 0.6266
Validation F1 for Hate Speech: 0.4688
Epoch 2/10


  0%|          | 0/574 [00:00<?, ?it/s]<ipython-input-7-c1faf5791dbd>:121: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 574/574 [01:40<00:00,  5.73it/s]


...

Average training loss: 0.1757
Validation F1 for Hate Speech: 0.4781
Epoch 10/10


  0%|          | 0/574 [00:00<?, ?it/s]<ipython-input-7-c1faf5791dbd>:121: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 574/574 [01:38<00:00,  5.80it/s]


Average training loss: 0.1505
Validation F1 for Hate Speech: 0.5111
Early stopping triggered.


<ipython-input-7-c1faf5791dbd>:162: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_rebert_model.pth"))


Test Metrics:
              precision    recall  f1-score   support

 Hate Speech       0.45      0.53      0.48       214
   Offensive       0.95      0.93      0.94      2879
     Neutral       0.87      0.91      0.89       625

    accuracy                           0.91      3718
   macro avg       0.76      0.79      0.77      3718
weighted avg       0.91      0.91      0.91      3718

